# Clustering Crypto

In [150]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [151]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
file = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file, index_col= 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [152]:
crypto_df.isnull().values.any()

True

In [153]:
crypto_df.shape

(1252, 6)

In [154]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
#crypto_df[crypto_df.IsTrading == "True" ]
crypto_df_filt = crypto_df.loc[crypto_df['IsTrading'] != False]

In [155]:
crypto_df_filt.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [156]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE

In [157]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
new_crypto_df = crypto_df_filt.drop(['IsTrading'], axis=1)
new_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [158]:
#Find null values
for column in new_crypto_df.columns:
        print(f"Column{column} has {new_crypto_df[column].isnull().sum()} null values")

ColumnCoinName has 0 null values
ColumnAlgorithm has 0 null values
ColumnProofType has 0 null values
ColumnTotalCoinsMined has 459 null values
ColumnTotalCoinSupply has 0 null values


In [159]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
new_crypto_df = new_crypto_df.dropna()
new_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [160]:
#Find null values
for column in new_crypto_df.columns:
        print(f"Column{column} has {new_crypto_df[column].isnull().sum()} null values")

ColumnCoinName has 0 null values
ColumnAlgorithm has 0 null values
ColumnProofType has 0 null values
ColumnTotalCoinsMined has 0 null values
ColumnTotalCoinSupply has 0 null values


In [161]:
# Keep the rows where coins are mined.
# YOUR CODE HERE
crypto_df_mined = new_crypto_df.loc[new_crypto_df['TotalCoinsMined'] > 0 ]
crypto_df_mined

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [162]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
crypto_df_coin = crypto_df_mined['CoinName']
crypto_df_coin

42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 532, dtype: object

In [163]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
crypto_df_new = crypto_df_mined.drop(['CoinName'], axis=1)
crypto_df_new

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [164]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
crypto_binary_encoded = pd.get_dummies(crypto_df_new, columns=['Algorithm', 'ProofType'])
crypto_binary_encoded.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
data_scaler = StandardScaler()

In [166]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE


crypto_data_scaled = data_scaler.fit_transform(crypto_binary_encoded)
crypto_data_scaled[:5]


array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [167]:
pca = PCA(n_components=3)

In [168]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
crypto_data_scaled_pca = pca.fit_transform(crypto_data_scaled)
crypto_data_scaled_pca

array([[-0.33873117,  1.04211418, -0.46193282],
       [-0.32205762,  1.04245948, -0.46227342],
       [ 2.31196722,  1.87231444, -0.50842126],
       ...,
       [ 0.34584867, -2.16490023,  0.29859576],
       [-0.13144147, -2.05115335,  0.3955564 ],
       [-0.31056036,  0.63007085, -0.21958246]])

In [169]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
pca_df = pd.DataFrame(
    data=crypto_data_scaled_pca, columns=["PC 1", "PC 2","PC 3"],index=crypto_binary_encoded.index
)
pca_df

,PC 1,PC 2,PC 3
42,-0.338731,1.042114,-0.461933
404,-0.322058,1.042459,-0.462273
1337,2.311967,1.872314,-0.508421
BTC,-0.140274,-1.276950,0.104604
ETH,-0.145959,-1.943536,0.316893
...,...,...,...
ZEPH,2.452287,0.485055,-0.247076
GAP,-0.336773,1.042042,-0.461952
BDX,0.345849,-2.164900,0.298596
ZEN,-0.131441,-2.051153,0.395556


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [170]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1, 11))

In [171]:
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

In [172]:
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [173]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=3, random_state=1)
# Fit the model
# YOUR CODE HERE
model.fit(pca_df)
# Predict clusters
# YOUR CODE HERE
predictions = model.predict(pca_df) 
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [174]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = pd.concat([crypto_df_new, pca_df],axis=1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df = pd.concat([clustered_df,crypto_df_coin],axis=1)
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
#class_df = pd.Series(predictions)
#clustered_df = pd.concat([clustered_df,class_df],axis=1) 
clustered_df ['Class'] = model.labels_
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.338731,1.042114,-0.461933,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322058,1.042459,-0.462273,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.311967,1.872314,-0.508421,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.140274,-1.276950,0.104604,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.145959,-1.943536,0.316893,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168207,-1.158412,-0.032960,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.375925,1.501046,-0.411049,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.134490,-2.135123,0.271937,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.144399,-1.943600,0.316879,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.131440,-2.051153,0.395556,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [175]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
#scat_df =px.data.clustered_df()
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    #hover_name="CoinName",
    color='Class',
    symbol='Class',
    width=800,
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [143]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [144]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
print(clustered_df)

        Algorithm ProofType  TotalCoinsMined TotalCoinSupply      PC 1  \
42         Scrypt   PoW/PoS     4.199995e+01              42 -0.333670   
404        Scrypt   PoW/PoS     1.055185e+09       532000000 -0.317026   
1337          X13   PoW/PoS     2.927942e+10    314159265359  2.302351   
BTC       SHA-256       PoW     1.792718e+07        21000000 -0.141877   
ETH        Ethash       PoW     1.076842e+08               0 -0.146719   
...           ...       ...              ...             ...       ...   
ZEPH      SHA-256      DPoS     2.000000e+09      2000000000  2.485009   
GAP        Scrypt   PoW/PoS     1.493105e+07       250000000 -0.331718   
BDX   CryptoNight       PoW     9.802226e+08      1400222610  0.324331   
ZEN      Equihash       PoW     7.296538e+06        21000000 -0.171572   
XBC        Scrypt       PoS     1.283270e+05         1000000 -0.288228   

          PC 2      PC 3     CoinName  Class  
42    1.071906 -0.567637      42 Coin      1  
404   1.071991 -0

In [145]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
total_coin_df = pd.DataFrame(
    data=clustered_df, columns=["TotalCoinSupply", "TotalCoinsMined"],index=crypto_binary_encoded.index
)
total_coin_df
data = total_coin_df
scaler = MinMaxScaler()
rescaled_clustered_df = scaler.fit_transform(total_coin_df)
rescaled_clustered_df

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [146]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df = pd.DataFrame(
    data=rescaled_clustered_df,columns=["TotalCoinSupply", "TotalCoinMined"],index=clustered_df.index
)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
coin_name = clustered_df["CoinName"]
plot_df = plot_df.join(coin_name)
plot_df
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
new_class = clustered_df["Class"]
plot_df = plot_df.join(new_class)


plot_df.head(10)

,TotalCoinSupply,TotalCoinMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [147]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(x="TotalCoinMined", y="TotalCoinSupply", by="Class")

:NdOverlay   [Class]
   :Scatter   [TotalCoinMined]   (TotalCoinSupply)